<a href="https://colab.research.google.com/github/GiriRaju45/ML-Algorithms-and-Coding-questions/blob/main/Bear%2C_Tiger_Classification_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# importing required libraries

import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import PIL
from PIL import Image
import tensorflow as tf
import pandas as pd
from tensorflow.keras.layers import Dropout, Dense, GlobalAveragePooling2D, Flatten
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, Model
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [ ]:
# creating dataset using given raw images for internet, pinterest, etc.

def create_dataset(img_folder):

    img_data_array=[]
    class_name=[]

    for dir1 in os.listdir(img_folder):
        for file in os.listdir(os.path.join(img_folder, dir1)):

            image_path= os.path.join(img_folder, dir1,  file)
            image= Image.open(image_path)
            image= image.convert('RGB')
            image = np.resize(image,(224,224,3))
            image= np.array(image)
            image = image.astype('float32')
            image /= 255
            img_data_array.append(image)
            class_name.append(dir1)
    return img_data_array, class_name
# extract the image array and class name
img_data, class_name =create_dataset('C:/image_data/')

In [ ]:
# generating new images using imagedatagenerator. -- Data Augmentation

datagen = ImageDataGenerator(
        rotation_range= 30, width_shift_range = 0.2, height_shift_range = 0.2, shear_range = 0.3, zoom_range = 0.4 , horizontal_flip = True, fill_mode = 'nearest')

In [ ]:
img = load_img('C:/image_data/tiger_images/110.luther-bostons-iconic-white-tiger-dies-at-14.jpg')
x = img_to_array(img)
print(x.shape)
x = x.reshape((1,) + x.shape)
print(x.shape)
i = 0
for batch in datagen.flow(x, batch_size=1, save_to_dir = 'C:/image_data/tiger_images/'
                         ,save_prefix = 'tiger', save_format = 'jpg' ):
    i+=1
    if i>20:

        break

(200, 200, 3)
(1, 200, 200, 3)


In [ ]:
# creating categories for the images and assigning labels
target_dict = {k:val for val,k in enumerate(np.unique(class_name))}
target_dict

{'Bear images': 0, 'tiger_images': 1}

In [ ]:
target_val=  [target_dict[class_name[i]] for i in range(len(class_name))]

In [ ]:
# using VGG19 MODEL FOR TRANSFER LEARNING
img_width,img_height = 224,224
model = tf.keras.applications.VGG19(include_top=False, input_shape= (img_width, img_height, 3), weights = 'imagenet')
model.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [ ]:
last_layer = model.get_layer('block5_pool')
layers = Model(inputs = model.inputs, outputs = last_layer.output)
layers.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [ ]:
# adding new layers to the pretrained model based on the application
new_model = Sequential()
new_model.add(layers)
new_model.add(Dropout(0.4))
new_model.add(Flatten())
new_model.add(Dense(1, activation = 'sigmoid'))
new_model.layers[0].trainable = False
new_model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001), loss  = tf.keras.losses.BinaryCrossentropy(), metrics = ['accuracy'])


In [ ]:
#Training model for 25 epochs


epochs= 25
hist = new_model.fit(x = np.array(img_data, np.float32),y =np.array(target_val), epochs = 25)

Epoch 1/25
33/33 [==============================] - 229s 7s/step - loss: 0.6529 - accuracy: 0.6336
Epoch 2/25
33/33 [==============================] - 239s 7s/step - loss: 0.4408 - accuracy: 0.7829
Epoch 3/25
33/33 [==============================] - 243s 7s/step - loss: 0.3789 - accuracy: 0.8390
Epoch 4/25
33/33 [==============================] - 245s 7s/step - loss: 0.3328 - accuracy: 0.8666
Epoch 5/25
33/33 [==============================] - 247s 7s/step - loss: 0.3461 - accuracy: 0.8624
Epoch 6/25
33/33 [==============================] - 249s 8s/step - loss: 0.3291 - accuracy: 0.8566
Epoch 7/25
33/33 [==============================] - 242s 7s/step - loss: 0.3266 - accuracy: 0.8463
Epoch 8/25
33/33 [==============================] - 240s 7s/step - loss: 0.2813 - accuracy: 0.8807
Epoch 9/25
33/33 [==============================] - 241s 7s/step - loss: 0.2664 - accuracy: 0.8922
Epoch 10/25
33/33 [==============================] - 245s 7s/step - loss: 0.2868 - accuracy: 0.8751
Epoch 11/

In [ ]:
#the above model gives accuracy of 93% for 25 epochs increasing epochs may increase the accuracy further more.